# Imports
All problem-speciffic files (code) in in the folder `HyperElasticity3D`, namely the loading of pre-prepared mesh and definition of energy in jax.numpy.

In [1]:
from tools.minimizers import newton
from tools.sparse_solvers import HessSolverGenerator
from tools.jax_diff import EnergyDerivator as EnergyDerivator
from HyperElasticity3D.jax_energy import J
from HyperElasticity3D.mesh import MeshHyperElasticity3D
from HyperElasticity3D.rotate_boundary import rotate_boundary

import numpy as np
from jax import config
from IPython.display import display, HTML
import pandas as pd

config.update("jax_enable_x64", True)

# Setup of the problem, Level = 1

In [2]:
rotation_per_iter = 4 * 2 * np.pi / 24
mesh = MeshHyperElasticity3D(mesh_level=1)
params, adjacency, u_init = mesh.get_data_jax()
energy_hyperelasticity = EnergyDerivator(J, params, adjacency, u_init)
print(f"Time for compilation: {energy_hyperelasticity.timings['compilation']:.4f}, time for coloring: {energy_hyperelasticity.timings['coloring']:.4f}")
print(f"Total setup time: {energy_hyperelasticity.timings['compilation'] + energy_hyperelasticity.timings['coloring']:.4f}")

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


Time for compilation: 0.3748, time for coloring: 0.0468
Total setup time: 0.4216


## Solution of 24 time steps

In [3]:
# Initialize the DataFrame to store results
results_df = pd.DataFrame(columns=['Time', 'Nit', 'Fun'])

for i in range(24):
    params = rotate_boundary(params, angle=rotation_per_iter)
    energy_hyperelasticity.params = params

    F, dF, ddF = energy_hyperelasticity.get_derivatives()
    ddf_with_solver = HessSolverGenerator(ddf=ddF, solver_type="direct", elastic_kernel=mesh.elastic_kernel, verbose=False, tol=1e-3)
    res = newton(F, dF, ddf_with_solver, u_init, verbose=False, tolf=1e-5, linesearch_tol=1e-3)
    u_init = res["x"]

    # Create a DataFrame for the current result, round the results, and concatenate it with the main DataFrame
    current_result_df = pd.DataFrame([{
        'Time': round(res['time'], 2),
        'Nit': res['nit'],
        'Fun': round(float(res['fun']), 4)
    }])
    results_df = pd.concat([results_df, current_result_df], ignore_index=True)

# Display the DataFrame as HTML in Jupyter Notebook
results_df.index = range(1,25)  # Reset index to simple counter
display(HTML(results_df.to_html()))

# Select every third row and print the DataFrame as LaTeX using the recommended style approach
latex_export_df = results_df.iloc[2::3]  # Slicing to select every third row
# Apply rounding for LaTeX export using Styler.format
latex_style = latex_export_df.style.format({
    'Time': "{:.2f}",
    'Fun': "{:.4f}"
})

print(latex_style.to_latex())


,Time,Nit,Fun
1,0.33,19,0.3464
2,0.25,19,1.3856
3,0.25,19,3.1173
4,0.25,19,5.5401
5,0.25,19,8.6505
6,0.26,20,12.4423
7,0.26,20,16.9116
8,0.30,23,22.0617
9,0.27,21,27.8990
10,0.27,21,34.4265


\begin{tabular}{lrlr}
 & Time & Nit & Fun \\
3 & 0.25 & 19 & 3.1173 \\
6 & 0.26 & 20 & 12.4423 \\
9 & 0.27 & 21 & 27.8990 \\
12 & 0.26 & 20 & 49.5501 \\
15 & 0.30 & 23 & 77.3831 \\
18 & 0.28 & 21 & 111.3262 \\
21 & 0.30 & 23 & 151.4552 \\
24 & 0.62 & 47 & 197.7484 \\
\end{tabular}



# Setup of the problem, Level = 2

In [7]:
rotation_per_iter = 4 * 2 * np.pi / 24
mesh = MeshHyperElasticity3D(mesh_level=2)
params, adjacency, u_init = mesh.get_data_jax()
energy_hyperelasticity = EnergyDerivator(J, params, adjacency, u_init)
print(f"Time for compilation: {energy_hyperelasticity.timings['compilation']:.4f}, time for coloring: {energy_hyperelasticity.timings['coloring']:.4f}")
print(f"Total setup time: {energy_hyperelasticity.timings['compilation'] + energy_hyperelasticity.timings['coloring']:.4f}")

Time for compilation: 0.8273, time for coloring: 0.8577
Total setup time: 1.6851


## Solution of 24 time steps

In [8]:
# Initialize the DataFrame to store results
results_df = pd.DataFrame(columns=['Time', 'Nit', 'Fun'])

for i in range(24):
    params = rotate_boundary(params, angle=rotation_per_iter)
    energy_hyperelasticity.params = params

    F, dF, ddF = energy_hyperelasticity.get_derivatives()
    ddf_with_solver = HessSolverGenerator(ddf=ddF, solver_type="direct", elastic_kernel=mesh.elastic_kernel, verbose=False, tol=1e-3)
    res = newton(F, dF, ddf_with_solver, u_init, verbose=False, tolf=1e-5, linesearch_tol=1e-3)
    u_init = res["x"]

    # Create a DataFrame for the current result, round the results, and concatenate it with the main DataFrame
    current_result_df = pd.DataFrame([{
        'Time': round(res['time'], 2),
        'Nit': res['nit'],
        'Fun': round(float(res['fun']), 4)
    }])
    results_df = pd.concat([results_df, current_result_df], ignore_index=True)

# Display the DataFrame as HTML in Jupyter Notebook
results_df.index = range(1,25)  # Reset index to simple counter
display(HTML(results_df.to_html()))

# Select every third row and print the DataFrame as LaTeX using the recommended style approach
latex_export_df = results_df.iloc[2::3]  # Slicing to select every third row
# Apply rounding for LaTeX export using Styler.format
latex_style = latex_export_df.style.format({
    'Time': "{:.2f}",
    'Fun': "{:.4f}"
})

print(latex_style.to_latex())


,Time,Nit,Fun
1,5.43,24,0.2027
2,4.22,21,0.8108
3,4.55,23,1.8244
4,4.62,24,3.2432
5,4.67,24,5.0673
6,4.70,24,7.2960
7,4.55,23,9.9290
8,4.20,22,12.9658
9,4.36,23,16.4069
10,4.41,23,20.2529


\begin{tabular}{lrlr}
 & Time & Nit & Fun \\
3 & 4.55 & 23 & 1.8244 \\
6 & 4.70 & 24 & 7.2960 \\
9 & 4.36 & 23 & 16.4069 \\
12 & 4.00 & 21 & 29.1607 \\
15 & 4.09 & 21 & 45.5598 \\
18 & 4.29 & 22 & 65.5437 \\
21 & 4.53 & 23 & 89.1459 \\
24 & 7.57 & 39 & 116.3232 \\
\end{tabular}



# Setup of the problem, Level = 3

In [11]:
rotation_per_iter = 4 * 2 * np.pi / 24
mesh = MeshHyperElasticity3D(mesh_level=3)
params, adjacency, u_init = mesh.get_data_jax()
energy_hyperelasticity = EnergyDerivator(J, params, adjacency, u_init)
print(f"Time for compilation: {energy_hyperelasticity.timings['compilation']:.4f}, time for coloring: {energy_hyperelasticity.timings['coloring']:.4f}")
print(f"Total setup time: {energy_hyperelasticity.timings['compilation'] + energy_hyperelasticity.timings['coloring']:.4f}")

Time for compilation: 0.5072, time for coloring: 4.3523
Total setup time: 4.8594


## Solution of 24 time steps

In [12]:
# Initialize the DataFrame to store results
results_df = pd.DataFrame(columns=['Time', 'Nit', 'Fun'])

for i in range(24):
    params = rotate_boundary(params, angle=rotation_per_iter)
    energy_hyperelasticity.params = params

    F, dF, ddF = energy_hyperelasticity.get_derivatives()
    ddf_with_solver = HessSolverGenerator(ddf=ddF, solver_type="amg", elastic_kernel=mesh.elastic_kernel, verbose=False, tol=1e-3)
    res = newton(F, dF, ddf_with_solver, u_init, verbose=False, tolf=1e-5, linesearch_tol=1e-3)
    u_init = res["x"]

    # Create a DataFrame for the current result, round the results, and concatenate it with the main DataFrame
    current_result_df = pd.DataFrame([{
        'Time': round(res['time'], 2),
        'Nit': res['nit'],
        'Fun': round(float(res['fun']), 4)
    }])
    results_df = pd.concat([results_df, current_result_df], ignore_index=True)

# Display the DataFrame as HTML in Jupyter Notebook
results_df.index = range(1,25)  # Reset index to simple counter
display(HTML(results_df.to_html()))

# Select every third row and print the DataFrame as LaTeX using the recommended style approach
latex_export_df = results_df.iloc[2::3]  # Slicing to select every third row
# Apply rounding for LaTeX export using Styler.format
latex_style = latex_export_df.style.format({
    'Time': "{:.2f}",
    'Fun': "{:.4f}"
})

print(latex_style.to_latex())


,Time,Nit,Fun
1,61.13,24,0.1626
2,67.77,23,0.6502
3,75.38,24,1.4631
4,72.63,24,2.6011
5,70.37,23,4.0645
6,77.67,24,5.8533
7,69.92,22,7.9675
8,73.90,23,10.4074
9,73.92,23,13.1731
10,67.97,21,16.2645


\begin{tabular}{lrlr}
 & Time & Nit & Fun \\
3 & 75.38 & 24 & 1.4631 \\
6 & 77.67 & 24 & 5.8533 \\
9 & 73.92 & 23 & 13.1731 \\
12 & 69.71 & 21 & 23.4252 \\
15 & 74.70 & 23 & 36.6100 \\
18 & 70.81 & 22 & 52.7169 \\
21 & 98.70 & 29 & 71.7516 \\
24 & 171.82 & 49 & 93.7039 \\
\end{tabular}

